# Section: Encrypted Deep Learning

- Lesson: Reviewing Additive Secret Sharing
- Lesson: Encrypted Subtraction and Public/Scalar Multiplication
- Lesson: Encrypted Computation in PySyft
- Project: Build an Encrypted Database
- Lesson: Encrypted Deep Learning in PyTorch
- Lesson: Encrypted Deep Learning in Keras
- Final Project

# Lesson: Encrypted Deep Learning in Keras


## Step 1: Public Training

Welcome to this tutorial! In the following notebooks you will learn how to provide private predictions. By private predictions, we mean that the data is constantly encrypted throughout the entire process. At no point is the user sharing raw data, only encrypted (that is, secret shared) data. In order to provide these private predictions, Syft Keras uses a library called [TF Encrypted](https://github.com/tf-encrypted/tf-encrypted) under the hood. TF Encrypted combines cutting-edge cryptographic and machine learning techniques, but you don't have to worry about this and can focus on your machine learning application.

You can start serving private predictions with only three steps:
- **Step 1**: train your model with normal Keras.
- **Step 2**: secure and serve your machine learning model (server).
- **Step 3**: query the secured model to receive private predictions (client). 

Alright, let's go through these three steps so you can deploy impactful machine learning services without sacrificing user privacy or model security.

Huge shoutout to the Dropout Labs ([@dropoutlabs](https://twitter.com/dropoutlabs)) and TF Encrypted ([@tf_encrypted](https://twitter.com/tf_encrypted)) teams for their great work which makes this demo possible, especially: Jason Mancuso ([@jvmancuso](https://twitter.com/jvmancuso)), Yann Dupis ([@YannDupis](https://twitter.com/YannDupis)), and Morten Dahl ([@mortendahlcs](https://github.com/mortendahlcs)). 

_Demo Ref: https://github.com/OpenMined/PySyft/tree/dev/examples/tutorials_

## Train Your Model in Keras

To use privacy-preserving machine learning techniques for your projects you should not have to learn a new machine learning framework. If you have basic [Keras](https://keras.io/) knowledge, you can start using these techniques with Syft Keras. If you have never used Keras before, you can learn a bit more about it through the [Keras documentation](https://keras.io). 

Before serving private predictions, the first step is to train your model with normal Keras. As an example, we will train a model to classify handwritten digits. To train this model we will use the canonical [MNIST dataset](http://yann.lecun.com/exdb/mnist/).

We borrow [this example](https://github.com/keras-team/keras/blob/master/examples/mnist_cnn.py) from the reference Keras repository.  To train your classification model, you just run the cell below.

In [1]:
from __future__ import print_function
import tensorflow.keras as keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, AveragePooling2D
from tensorflow.keras.layers import Activation

batch_size = 128
num_classes = 10
epochs = 2

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()

model.add(Conv2D(10, (3, 3), input_shape=input_shape))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

W0808 21:59:00.308102 12656 deprecation.py:506] From d:\ProgramData\Miniconda3\envs\pysyft\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 10s 159us/sample - loss: 2.3050 - acc: 0.1133 - val_loss: 2.3031 - val_acc: 0.1292
Epoch 2/2
60000/60000 [==============================] - 10s 158us/sample - loss: 2.3020 - acc: 0.1354 - val_loss: 2.3001 - val_acc: 0.1470
Test loss: 2.3001097480773924
Test accuracy: 0.147


In [2]:
## Save your model's weights for future private prediction
model.save('./models/short-conv-mnist.h5')

## Step 2: Load and Serve the Model

Now that you have a trained model with normal Keras, you are ready to serve some private predictions. We can do that using Syft Keras.

To secure and serve this model, we will need three TFEWorkers (servers). This is because TF Encrypted under the hood uses an encryption technique called [multi-party computation (MPC)](https://en.wikipedia.org/wiki/Secure_multi-party_computation). The idea is to split the model weights and input data into shares, then send a share of each value to the different servers. The key property is that if you look at the share on one server, it reveals nothing about the original value (input data or model weights).

We'll define a Syft Keras model like we did in the previous notebook. However, there is a trick: before instantiating this model, we'll run `hook = sy.KerasHook(tf.keras)`. This will add three important new methods to the Keras Sequential class:
 - `share`: will secure your model via secret sharing; by default, it will use the SecureNN protocol from TF Encrypted to secret share your model between each of the three TFEWorkers. Most importantly, this will add the capability of providing predictions on encrypted data.
 - `serve`: this function will launch a serving queue, so that the TFEWorkers can can accept prediction requests on the secured model from external clients.
 - `shutdown_workers`: once you are done providing private predictions, you can shut down your model by running this function. It will direct you to shutdown the server processes manually if you've opted to manually manage each worker.

If you want learn more about MPC, you can read this excellent [blog](https://mortendahl.github.io/2017/04/17/private-deep-learning-with-mpc/).

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import AveragePooling2D, Conv2D, Dense, Activation, Flatten, ReLU, Activation

import syft as sy
hook = sy.KerasHook(tf.keras)

W0808 21:59:20.612045 12656 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was 'd:\ProgramData\Miniconda3\envs\pysyft\lib\site-packages\tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0808 21:59:20.625036 12656 deprecation_wrapper.py:119] From d:\ProgramData\Miniconda3\envs\pysyft\lib\site-packages\tf_encrypted\session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



## Model

As you can see, we define almost the exact same model as before, except we provide a `batch_input_shape`. This allows TF Encrypted to better optimize the secure computations via predefined tensor shapes. For this MNIST demo, we'll send input data with the shape of (1, 28, 28, 1). 
We also return the logit instead of softmax because this operation is complex to perform using MPC, and we don't need it to serve prediction requests.

In [4]:
num_classes = 10
input_shape = (1, 28, 28, 1)

model = Sequential()

model.add(Conv2D(10, (3, 3), batch_input_shape=input_shape))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(num_classes, name="logit"))

### Load Pre-trained Weights

With `load_weights` you can easily load the weights you have saved previously after training your model.

In [5]:
pre_trained_weights = './models/short-conv-mnist.h5'
model.load_weights(pre_trained_weights)

## Step 3: Setup Your Worker Connectors

Let's now connect to the TFEWorkers (`alice`, `bob`, and `carol`) required by TF Encrypted to perform private predictions. For each TFEWorker, you just have to specify a host.

These workers run a [TensorFlow server](https://www.tensorflow.org/api_docs/python/tf/distribute/Server), which you can either manage manually (`AUTO = False`) or ask the workers to manage for you (`AUTO = True`). If choosing to manually manage them, you will be instructed to execute a terminal command on each worker's host device after calling `model.share()` below.  If all workers are hosted on a single device (e.g. `localhost`), you can choose to have Syft automatically manage the worker's TensorFlow server.

In [6]:
AUTO = False

alice = sy.TFEWorker(host='localhost:4000', auto_managed=AUTO)
bob = sy.TFEWorker(host='localhost:4001', auto_managed=AUTO)
carol = sy.TFEWorker(host='localhost:4002', auto_managed=AUTO)

## Step 4: Split the Model Into Shares

Thanks to `sy.KerasHook(tf.keras)` you can call the `share` method to transform your model into a TF Encrypted Keras model.

If you have asked to manually manage servers above then this step will not complete until they have all been launched. Note that your firewall may ask for Python to accept incoming connection.

In [7]:
model.share(alice, bob, carol)

W0808 21:59:21.143901 12656 deprecation_wrapper.py:119] From d:\ProgramData\Miniconda3\envs\pysyft\lib\site-packages\tf_encrypted\keras\engine\base_layer_utils.py:29: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0808 21:59:21.145898 12656 deprecation_wrapper.py:119] From d:\ProgramData\Miniconda3\envs\pysyft\lib\site-packages\tf_encrypted\tensor\native.py:403: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0808 21:59:21.150895 12656 deprecation_wrapper.py:119] From d:\ProgramData\Miniconda3\envs\pysyft\lib\site-packages\tf_encrypted\tensor\native.py:101: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0808 21:59:21.188868 12656 deprecation.py:506] From d:\ProgramData\Miniconda3\envs\pysyft\lib\site-packages\tf_encrypted\tensor\shared.py:62: calling extract_image_patches (from tensorflow.python.ops.array_ops) with ksizes is deprecated and will be removed in a fut

## Step 5: Launch 3 Servers

```
python -m tf_encrypted.player --config /tmp/tfe.config server0
python -m tf_encrypted.player --config /tmp/tfe.config server1
python -m tf_encrypted.player --config /tmp/tfe.config server2```

### For windows: 

`
python -m tf_encrypted.player --config %USERPROFILE%\AppData\Local\Temp/tfe.config server0
python -m tf_encrypted.player --config %USERPROFILE%\AppData\Local\Temp/tfe.config server1
python -m tf_encrypted.player --config %USERPROFILE%\AppData\Local\Temp/tfe.config server2
`


## Step 6: Serve the Model

Perfect! Now by calling `model.serve`, your model is ready to provide some private predictions. You can set `num_requests` to set a limit on the number of predictions requests served by the model; if not specified then the model will be served until interrupted.

In [8]:
model.serve(num_requests=3)

Served encrypted prediction 1 to client.
Served encrypted prediction 2 to client.
Served encrypted prediction 3 to client.


## Step 7: Run the Client

At this point open up and run the companion notebook: Section 4b - Encrytped Keras Client

## Step 8: Shutdown the Servers

Once your request limit above, the model will no longer be available for serving requests, but it's still secret shared between the three workers above. You can kill the workers by executing the cell below.

**Congratulations** on finishing Part 12: Secure Classification with Syft Keras and TFE!

In [ ]:
import os
import subprocess as sb
import psutil

model.shutdown_workers()

pids = []
if not AUTO:
    pids.append(os.popen('netstat -ano | findstr :4000').read().split()[4])
    pids.append(os.popen('netstat -ano | findstr :4001').read().split()[4])
    pids.append(os.popen('netstat -ano | findstr :4002').read().split()[4])
    print(pids)

    for pid in pids:
        p = psutil.Process(int(pid))
        p.terminate()
#         sb.Popen("taskkill /F /T /PID %i" % int(pid) , shell=True) #access denied


        

# Keystone Project - Mix and Match What You've Learned

Description: Take two of the concepts you've learned about in this course (Encrypted Computation, Federated Learning, Differential Privacy) and combine them for a use case of your own design. Extra credit if you can get your demo working with [WebSocketWorkers](https://github.com/OpenMined/PySyft/tree/dev/examples/tutorials/advanced/websockets-example-MNIST) instead of VirtualWorkers! Then take your demo or example application, write a blogpost, and share that blogpost in #general-discussion on OpenMined's slack!!!

Inspiration:
- This Course's Code: https://github.com/Udacity/private-ai
- OpenMined's Tutorials: https://github.com/OpenMined/PySyft/tree/dev/examples/tutorials
- OpenMined's Blog: https://blog.openmined.org